# Cats and dogs

In [27]:
include("generate_artifacts.jl")
data_dir = "../data"
artifact_name = "cats_dogs";

### Generate toy data

In [24]:
using Random
Random.seed!(1234)
averages = (dog=[100;20],cat=[50;50])
D = 2
N = 1000
y = rand([0,1],N)
dog_label = 1
cat_label = 0
X = zeros(D,N)
dogs = findall(y.==dog_label)
cats = findall(y.==cat_label)
noise = 0.1
noise_cats = randn(D,length(cats)) .* noise .*averages.cat 
noise_dogs = randn(D,length(dogs)) .* noise .*averages.dog 
X[:,cats] .= averages.cat .+ noise_cats
X[:,dogs] .= averages.dog .+ noise_dogs
data = Dict(
        "X" => X,
        "y" => y
    )
using BSON: @save
datafile = artifact_name * "_data.bson"
@save joinpath(data_dir, datafile) data

In [28]:
datafiles = [datafile]
generate_artifact(datafiles)

┌ Warning: For automatic github deployment, need GITHUB_TOKEN. Not found in ENV, attemptimg global git config.
└ @ Main /Users/patrickaltmeyer/Documents/code/AlgorithmicRecourse.jl/dev/artifacts/generate_artifacts.jl:13
┌ Info: Binding cats_dogs_data in Artifacts.toml...
└ @ Main /Users/patrickaltmeyer/Documents/code/AlgorithmicRecourse.jl/dev/artifacts/generate_artifacts.jl:57
┌ Info: Uploading tarballs to pat-alt/AlgorithmicRecourse.jl tag `data`
└ @ Main /Users/patrickaltmeyer/Documents/code/AlgorithmicRecourse.jl/dev/artifacts/generate_artifacts.jl:68


--> Deleting: cats_dogs_data.tar.gz
--> Uploading: cats_dogs_data.tar.gz


┌ Info: Artifacts.toml file now contains all bound artifact names
└ @ Main /Users/patrickaltmeyer/Documents/code/AlgorithmicRecourse.jl/dev/artifacts/generate_artifacts.jl:74
